# Two-particle self consistency (TPSC) with Triqs

This notebook is a rewrite of material from Prof. A.-M.S. Tremblay on the two-particle self consistency approach to the Hubbard model on the square lattice.

Author: Hugo U. R. Strand (2018)

# System parameters

Inverse temperature $\beta$, chemical potential $\mu$, and nearest neighbour hopping $t$.

The momentum space discretization is given by the number of $k$-points along each $k$-space basis vector $n_k = (n_{k_x}, n_{k_y}, n_{k_z})$ and we use a finite number $n_\omega$ of Matsubara frequencies.

In [1]:
beta = 1./0.4
mu = 0.0
t = 1.0 

n_k = (8, 8, 1) 
n_w = 128

# Two dimensional square lattice tight binding Hamiltonian

The non-interacting part of the Hamiltonian $H_0$ comprises of nearest neighbour hopping on the two dimensional square lattice

$$H_0 = -t \sum_{\langle i, j\rangle, \sigma} [
c^\dagger_{i\sigma} c_{j\sigma} + c^\dagger_{j\sigma} c_{i\sigma} ]$$

where $c^\dagger_{i\sigma}$ creates an electron on site $i$ with spin $\sigma \in \{ \uparrow, \downarrow \}$.

In [2]:
%matplotlib inline
import numpy as np
from triqs.lattice.tight_binding import TBLattice

I = np.eye(2)

H_0 = TBLattice(
    units = [(1, 0, 0), (0, 1, 0)],
    hopping = {
        (+1, 0): -t * I, # hopping in +x 
        (-1, 0): -t * I, # hopping in -x
        ( 0,+1): -t * I, # hopping in +y
        ( 0,-1): -t * I, # hopping in -y
        },
    orbital_positions = [(0,0,0)]*2,
    orbital_names = ['up', 'do'],
    )

Starting on 1 Nodes at : 2018-05-15 14:13:20.036428


# Momentum space dispersion relation $\epsilon_k$

For the numerical calculations we represent the hopping Hamiltoninan $H_0$ on a discretized mesh in momentum space `kmesh` and compute the Fourier transform of the real-space hopping to momentum space, giving the $k$-space dispersion $\epsilon_k$.

$$H_0 = \sum_{k\sigma} \epsilon_k c^\dagger_{k\sigma} c_{k\sigma}$$

In [3]:
            e_k = H_0.on_gf_mesh_brillouin_zone(n_k)
print 'e_k =\n', e_k

kmesh = e_k.mesh
bz = kmesh.domain

from triqs.lattice import BravaisLattice
bl = BravaisLattice(bz)

from triqs.gf import MeshCyclicLattice
periodization_matrix = np.array(np.diag(kmesh.linear_dims), dtype=np.int32)

rmesh = MeshCyclicLattice(bl, periodization_matrix)

e_k =
Green Function  with mesh Brillouin Zone Mesh of size (8 8 1), Domain: Brillouin Zone with repiprocal matrix 
[[6.28319,0,0]
 [0,6.28319,0]
 [0,0,6.28319]] and target_rank 2: 



AttributeError: 'triqs.gf.meshes.MeshBrillouinZone' object has no attribute 'domain'

In [ ]:
from triqs.plot.mpl_interface import plt
from k_space_viz import oplot_bz_cl_mesh_2d

plt.figure(figsize=(3.25*3, 8))
plt.title(r'Dispersion $\epsilon_k$ and fermi surface contour $\epsilon_k = 0$')

oplot_bz_cl_mesh_2d(e_k[0, 0], levels=[0.])

plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')
plt.colorbar(); plt.tight_layout()
plt.savefig('figure_e_k.pdf')

In [ ]:
from triqs.plot.mpl_interface import plt
from k_space_viz import oplot_bz_path

plt.figure(figsize=(3.25*3, 8))
plt.title(r'Dispersion $\epsilon_k$ along the high-symmetry path $\Gamma$-$X$-$M$-$\Gamma$')

G = np.array([0.0, 0.0, 0.0])
X = np.array([0.5, 0.5, 0.0])
M = np.array([0.5, 0.0, 0.0])

paths = [(G, X), (X, M), (M, G)]

oplot_bz_path(e_k[0, 0], paths)

plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$'])
plt.ylabel(r'$\epsilon_k$'); plt.tight_layout()
plt.savefig('figure_e_k_bandpath.pdf')

# Real-space hopping $t_{i - j}$ from $\epsilon_k$

The dispersion relation $\epsilon_k$ is a complete description of the real-space hopping $t_{i - j} = -t \delta_{\langle i, j \rangle}$ and as a sanity check we can Fourier trasform back to real space and plot the hopping. 

In [ ]:
rmesh = H_0.get_rmesh(n_k)

from triqs.gf import Gf, InverseFourier
t_r = Gf(mesh=rmesh, target_shape=e_k.target_shape)
t_r << InverseFourier(e_k)

In [ ]:
from triqs.plot.mpl_interface import plt
from k_space_viz import oplot_bz_cl_mesh_2d

plt.figure(figsize=(3.25*3, 8))
plt.title(r'Dispersion $\epsilon_k$ and fermi surface contour $\epsilon_k = 0$')

oplot_bz_cl_mesh_2d(t_r[0, 0], cmap=plt.get_cmap('terrain'))

plt.xlabel(r'$r_x$')
plt.ylabel(r'$r_y$')
plt.colorbar(); plt.tight_layout()
plt.savefig('figure_e_r.pdf')

# Single particle Green's function

The Matsubara frequency single-particle Green's function for the non-interacting system is given directly by the dispersion relation as

$$g_0(i\omega_n, k) \equiv \big[ i\omega_n - \mu - \epsilon_k \big]^{-1}$$

we also compute its inverse Fourier transform $\mathcal{F}^{-1} \{ \cdot \}$ back to real space $r$ and imaginary time $\tau$, i.e.

$$ g_0(i\omega_n, r) \equiv \mathcal{F}^{-1}_{k \rightarrow r} \big\{ g_0(i\omega_n, k) \big\}$$

$$ g_0(\tau, r) \equiv \mathcal{F}^{-1}_{i\omega_n \rightarrow \tau} \big\{ g_0(i\omega_n, r) \big\}$$

Similarly as for real- and momentum-space we define discretized meshes in Matsubara frequency `wmesh` and imaginary time `tmesh` on which we store the Green's function.

In [ ]:
from triqs.gf import Gf
#from triqs.gf import TailGf
from triqs.gf import inverse, iOmega_n, Idx
from triqs.gf import Fourier, InverseFourier
from triqs.gf import MeshImFreq, MeshImTime, MeshProduct

In [ ]:
# todo: remove when we have the new tail

def fix_tail(g_wr, r):
    g_w = g_wr[:, r]
    g_w.singularity = TailGf(g_w.target_shape)
    if np.linalg.norm(r.value) < 1e-9: g_w.singularity[1] = I
    return g_w

In [ ]:
def get_g0(e_k, beta, mu, n_w, kmesh, rmesh):
    
    target_shape = e_k.target_shape

    wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)
    tmesh = MeshImTime(beta=beta, S='Fermion', n_max=2*len(wmesh))

    g0_wk = Gf(mesh=MeshProduct(wmesh, kmesh), target_shape=target_shape)
    g0_wr = Gf(mesh=MeshProduct(wmesh, rmesh), target_shape=target_shape)
    g0_tr = Gf(mesh=MeshProduct(tmesh, rmesh), target_shape=target_shape)

    for k in kmesh: g0_wk[:, k] = inverse( iOmega_n + mu * I - e_k[k] )
    for w in wmesh: g0_wr[w, :] = InverseFourier(g0_wk[w, :])
    #for r in rmesh: g0_tr[:, r] = InverseFourier(fix_tail(g0_wr, r))
    for r in rmesh: g0_tr[:, r] = InverseFourier(g0_wr[:, r])

    return g0_wk, g0_wr, g0_tr

In [ ]:
g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)

In [ ]:
from triqs.plot.mpl_interface import oplot, oplotr, oploti, plt
plt.figure()
oplot(g0_tr[:, Idx(0, 0, 0)])

# Momentum distribution at $\epsilon_{F}$, i.e. $-\mathrm{Im}[G(\omega=0, k)]/\pi$

In [ ]:
from triqs.plot.mpl_interface import oplot, oplotr, oploti, plt

g0_w0k = Gf(mesh=kmesh, target_shape=e_k.target_shape)
for k in g0_w0k.mesh: g0_w0k[k] = 1./( (1.e-1j + mu) * I - e_k[k] )
    
data, k_vec, (kx, ky, kz) = extend_data_on_boundary(g0_w0k[0, 0].data, n_k)
data = -data[:, :, 0].imag / np.pi
extent_k = np.array([kx.min(), kx.max(), ky.min(), ky.max()])

plt.figure(figsize=(3.25*4, 5*2))
plt.imshow(data, cmap=plt.get_cmap('magma'), origin='lower', 
           vmin=0, vmax=data.max(), extent=extent_k)
plt.colorbar(); plt.tight_layout()
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')
plt.savefig('figure_ef_momentum_distribution.pdf')

# Real space imaginary time Green's function

To get some intuition about the real-space imaginary time Green's function we plot the local, the nearest neighbour, the next nearest neighbour, and next next nearest neighbour components.

Q: Can you predict how $g_0(\tau, [0,1,0])$ looks like? What about $g_0(\tau, [1,1,1])$?

In [ ]:
from triqs.plot.mpl_interface import oplot, oplotr, oploti, plt

plt.figure(figsize=(3.25*4, 8))

oplotr(g0_tr[:, Idx(0, 0, 0)][0, 0], label=r'$g_0(\tau, [0,0,0])$')
oplotr(g0_tr[:, Idx(1, 0, 0)][0, 0], label=r'$g_0(\tau, [1,0,0])$')
oplotr(g0_tr[:, Idx(1, 1, 0)][0, 0], label=r'$g_0(\tau, [1,1,0])$')
oplotr(g0_tr[:, Idx(2, 0, 0)][0, 0], label=r'$g_0(\tau, [2,0,0])$')
oplotr(g0_tr[:, Idx(2, 1, 0)][0, 0], label=r'$g_0(\tau, [2,1,0])$')

# Lindhardt susceptibility $\chi_0$

The two-particle response of the non-interacting system is given by the Lindhardt susceptibility $\chi_0$. Diagrammatically it is a bubble diagram of two single-particle Green's function, that is most efficiently evaluated in imaginary time.

$$ \chi_0(\tau, r) \equiv 2 g_0(\tau, r) g_0(\beta - \tau, -r) $$

after evaluating the imaginary time bubble we transform $\chi_0$ back to momentum and frequency space, i.e.


$$ \chi_0(i\omega_n, r) \equiv \mathcal{F}_{\tau \rightarrow i\omega_n} \big\{ \chi_0(\tau, r) \big\}$$

$$ \chi_0(i\omega_n, k) \equiv \mathcal{F}_{r \rightarrow k} \big\{ \chi_0(i\omega_n, r) \big\}$$


In [ ]:
%reload_ext cpp2py.magic

Code C++ from Olivier

In [ ]:
%%cpp2py -C triqs
#include <triqs/gfs.hpp> 
using namespace triqs::gfs;

// The type of a Green function : (omega,k) -> Complex number
using g_tau_k_type = gf_view<cartesian_product<imfreq, brillouin_zone>>;

g_tau_k_type bubble(g_tau_k_type g) { 

    // Fourier Transformation of k, \omega to obtain g(t,r)
    auto gtr = make_gf_from_fourier<0,1>(g);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mt, mr] = gtr.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (tau, r) with this new mesh.
    auto chi0 = gf<cartesian_product<imtime, cyclic_lattice>>{{mtb, mr}};

    // we fill chi : chi(tau, r) = g(beta - tau, -r) * g(tau, r)
    for (auto const &r : mr)      
        for (auto const &t : mtb) 
            chi0[t, r] = gtr(beta - t, -r) * gtr(t, r); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

Code C++ from Hugo

In [ ]:
%%cpp2py -C triqs
 
#include <triqs/gfs.hpp> 
using namespace triqs::gfs;

using mesh = cartesian_product<imtime, cyclic_lattice>;
using g_tr_t = gf<mesh, matrix_valued>;
using g_tr_cvt = g_tr_t::const_view_type;

gf_mesh<imtime> operator*(gf_mesh<imtime> A, gf_mesh<imtime> B) {
    auto SA = A.domain().statistic;
    auto SB = B.domain().statistic;
    return gf_mesh<imtime>{A.domain().beta, SA * SB, A.size()};
}

int sign(gf_mesh<imtime> mesh) { return sign(mesh.domain().statistic); }

    
    auto SL = tmeshL.domain().statistic;
    auto SR = tmeshR.domain().statistic;
    auto tmesh = gf_mesh<imtime>{beta, SL * SR, tmeshL.size()};

    

g_tr_t bubble(g_tr_cvt gL_tr, g_tr_cvt gR_tr) {

    auto [tmeshL, rmeshL] = gL_tr.mesh();
    auto [tmeshR, rmeshR] = gR_tr.mesh();
    
    auto beta = tmeshL.domain().beta;
    auto signR = sign(tmeshR);
    
    g_tr_t chi0{{tmeshL * tmeshR, rmeshL}, {1, 1}};

    for(auto const &[t, r] : chi0.mesh())
        chi0[t, r] = - gL_tr(t, r) * signR * gR_tr(beta - t, -r);
    
    return chi0;
}

In [ ]:
def reverse_tau(g_tr):
    g_tr_out = g_tr.copy()
    g_tr_out.data[:] = g_tr.data[::-1, ...]
    return g_tr_out

def reverse_r(g_tr):
    data = g_tr.data.copy()
    rmesh = g_tr.mesh.components[1]
    
    shape = data.shape
    new_shape = list(shape[:1]) + list(rmesh.linear_dims) + list(shape[2:])    

    data = data.reshape(new_shape)[:, ::-1, ::-1, ::-1, ...]
    data = np.roll(data, [1, 1, 1], axis=[1, 2, 3])
    data = data.reshape(shape)
    
    g_tr_out = g_tr.copy()
    g_tr_out.data[:] = data
    return g_tr_out

def python_bubble(gL_tr, gR_tr, n_k, n_w, beta):
    tmesh = MeshImTime(beta=beta, S='Boson', n_max=len(gL_tr.mesh.components[0]))
    chi0_tr = Gf(mesh=MeshProduct(tmesh, rmesh), target_shape=[1, 1])
    chi0_tr << 2. * gL_tr[0,0] * reverse_r(reverse_tau(gR_tr[0,0]))
    return chi0_tr

In [ ]:
def get_chi0(g0_tr, n_w, kmesh, rmesh):

    target_shape = [1, 1]
    wmesh = MeshImFreq(beta=beta, S='Boson', n_max=n_w)
    tmesh = MeshImTime(beta=beta, S='Fermion', n_max=4*len(wmesh))

    chi0_tr = Gf(mesh=MeshProduct(tmesh, rmesh), target_shape=target_shape)
    chi0_wr = Gf(mesh=MeshProduct(wmesh, rmesh), target_shape=target_shape)
    chi0_wk = Gf(mesh=MeshProduct(wmesh, kmesh), target_shape=target_shape)

    chi0_tr = 2. * bubble(g0_tr, g0_tr)
    
    chi0_tr_ref = python_bubble(g0_tr.copy(), g0_tr.copy(), n_k, n_w, beta)
    np.testing.assert_array_almost_equal(chi0_tr.data, chi0_tr_ref.data)
    
    #for r in rmesh: chi0_wr[:, r] = Fourier(fix_tail(chi0_tr, r))
    for r in rmesh: chi0_wr[:, r] = Fourier(chi0_tr[: ,r])
    for w in wmesh: chi0_wk[w, :] = Fourier(chi0_wr[w, :])

    return chi0_tr, chi0_wr, chi0_wk

# Static Lindhardt susceptibility $\chi_0(\omega=0, k)$

The square lattice with only nearest-neighbour hopping $t$ has a property of "perfect nesting", meaning that large parts of the fermi surface are wrapped on-to each other by a single momentum transfer $k$. Go back to the fermi-surface polot of $\epsilon_k$ and determine this peculiar momentum vector.

The "perfect nesting" greatly enhances the particle-hole susceptibility of the system and the Static Lindhardt susceptibility $\chi_0(\omega=0, k)$ has a dominant peak at this momentum.

In [ ]:
chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)

In [ ]:
from triqs.plot.mpl_interface import oplot, oplotr, oploti, plt

chi0_w0k = chi0_wk[Idx(0), :][0, 0]
data, k_vec, (kx, ky, kz) = extend_data_on_boundary(chi0_w0k.data, n_k)
extent_k = np.array([kx.min(), kx.max(), ky.min(), ky.max()])

plt.figure(figsize=(3.25*4, 5*2))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')
plt.imshow(data[:, :, 0].real, cmap=plt.get_cmap('terrain_r'), origin='lower', 
           vmin=0, extent=extent_k)
plt.colorbar(); plt.tight_layout()
plt.savefig('figure_chi0_w0k.pdf')
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')

In [ ]:
from k_space_viz import get_rel_k_interpolator

interp = get_rel_k_interpolator(
    chi0_wk[Idx(0), :][0, 0].data, kmesh, H_0.bz, n_k,
    extend_boundary=True, interpolator='linear2D')
chi0_plot = interp(k_path_vecs[:, :2])

from triqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')
plt.plot(k_plot, chi0_plot.real)

plt.grid(); plt.axes().set_xticks(K_plot)
plt.xlim([K_plot.min(), K_plot.max()])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$'])
plt.ylabel(r'$\epsilon_k$'); plt.tight_layout()
plt.savefig('figure_chi0_k_bandpath.pdf')

# Bethe-Salpeter equation (BSE) and the random phase approximation (RPA)

The Lindhardt susceptibility $\chi_0$ is the exeact susceptibility for the non-interacting case $U=0$, however for finite interactions the susceptibility of the system $\chi$ is given by the Behte-Salpeter equation

$$ \chi = \chi_0 + \chi_0 \Gamma \chi $$

where $\Gamma$ is the particle-hole irreducible vertex function, containing all diagrams with insertions of the interaction that can not be separated by cutting a pair of particle-hole single-particle propagators $G G$.

The first order contribution to the vertext $\Gamma$ is the bare interaction $U$ and the approximation

$$ \Gamma \approx U $$

gives the so-called random phase approximation for $\chi$, i.e.

$$ \chi_{RPA} = \chi_0 + \chi_0 U \chi_{RPA} $$

Rewriting this equation gives $\chi_{RPA}$ as

$$ \chi_{RPA} = \frac{\chi_0}{1 - U \chi_0} $$

we note that the denominator of this equation can in general go to zero, whereby the susceptibility $\chi_{RPA}$ diverges. Whence already the RPA approximation can be used to compute instabilities of the system towards, e.g., anti-ferromagnetic symmetry breaking.

As an example we compute $\chi_{RPA}$ for the square lattice and the enhancement of the $k = (\pi, \pi)$ peak as a function of $U$.

In [ ]:
def chi_wk_from_U_and_chi0_wk(chi0_wk, U):
    U = float(U)
    chi_wk = chi0_wk.copy()
    kmesh = chi0_wk.mesh.components[1]
    for k in kmesh:
        chi_wk[:, k] << inverse(1. - U * chi0_wk[:, k]) * chi0_wk[:, k]

    return chi_wk

In [ ]:
from k_space_viz import get_rel_k_interpolator

interp = get_rel_k_interpolator(
    chi0_wk[Idx(0), :][0, 0].data, kmesh, H_0.bz, n_k,
    extend_boundary=True, interpolator='linear2D')
chi0_plot = interp(k_path_vecs[:, :2])

from triqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')

plt.plot(k_plot, chi0_plot.real, label=r'$\chi_0$')

U_vec = np.arange(0.2, 1.2, 0.2)

for U in U_vec:
    print('U =', U)
    chi_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, U)
    
    interp = get_rel_k_interpolator(
        chi_wk[Idx(0), :][0, 0].data, kmesh, H_0.bz, n_k,
        extend_boundary=True, interpolator='linear2D')
    chi_plot = interp(k_path_vecs[:, :2])

    plt.plot(k_plot, chi_plot.real, label=r'$\chi_{RPA}$, $U=%2.2f$' % U)

plt.grid(); plt.axes().set_xticks(K_plot)
plt.xlim([K_plot.min(), K_plot.max()])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$'])
plt.ylabel(r'$\epsilon_k$'); plt.tight_layout()
plt.legend(loc='best')
plt.savefig('figure_chi0_k_bandpath.pdf')

At some critical $U_c$ the susceptibility diverges $\chi \rightarrow \infty$ within the random phase approximation. To determine $U_c$ we can study the root of the inverse susceptibility $\chi_{RPA}^{-1}$.

For the square lattice it is sufficient to study the response at $k_{AF}= (\pi, \pi)$ since it is the point where the response diverges. Analytically this occurs when the denominator is zero $1 - U_c \chi_0(0, k_{AF}) = 0$, i.e.

$$ U_c^{(RPA)} = \frac{1}{\chi_0(0, k_{AF})} $$

numerically this looks like

In [ ]:
# find kidx where k[kidx] = (pi, pi, 0)
k_vec = np.array([k.value for k in kmesh])
diff = np.linalg.norm(k_vec - np.array([np.pi, np.pi, 0]), axis=1)
kidx_pipi = np.squeeze(np.argwhere(diff < 1.e-8))
k_pipi = np.array([k for k in kmesh])[kidx_pipi]
print('kidx_pipi =', kidx_pipi)
print('k_pipi =', k_pipi)

In [ ]:
chi0_w0kpipi = chi0_wk[Idx(0), k_pipi][0, 0].real

U_c = 1. / chi0_w0kpipi

U_vec = np.arange(0.2, 2.2, 0.2)

chi_vec = np.zeros_like(U_vec)
for idx, U in enumerate(U_vec):
    chi_vec[idx] = chi0_w0kpipi / (1 - U*chi0_w0kpipi)

In [ ]:
plt.figure(figsize=(3.25*3, 8))
plt.plot(U_vec, 1./chi_vec, '.-', label=r'$\chi_{RPA}^{-1}$')
plt.plot(U_vec, 0*U_vec, 'k', lw=0.5)
plt.plot(U_c, 0, 'rs', label=r'$U_c \approx %2.2f$' % U_c, alpha=0.5)
plt.legend(loc='best'); plt.xlim([U_vec.min(), U_vec.max()])
plt.ylabel(r'$\chi_{RPA}^{-1}$'); plt.xlabel(r'$U$')

# Two-particle self consistency (TPSC)

$$ \chi_{sp}(i\omega_n, k) \equiv \frac{\chi_0(i\omega_n, k)}{1 - \frac{U_{sp}}{2} \chi_0(i\omega_n, k)} $$

$$ \chi_{ch}(i\omega_n, k) \equiv \frac{\chi_0(i\omega_n, k)}{1 + \frac{U_{ch}}{2} \chi_0(i\omega_n, k)} $$

$$ \mathrm{Tr} [ \chi ] \equiv \frac{1}{\beta N_k} \sum_{n, k} \chi(i\omega_n, k) $$

In [ ]:
def chi_wk_trace(chi_wk):
    wmesh, kmesh = chi_wk.mesh.components
    tr_chi_wk = chi_wk.data.sum() / len(kmesh) / wmesh.beta # tail correction FIXME!!!
    assert(np.abs(tr_chi_wk.imag) < 1e-10)
    return tr_chi_wk.real

def Usp_root_problem(Usp, chi0, n, U):

    tr_chi_sp = chi_wk_trace(chi_wk_from_U_and_chi0_wk(chi0, U=0.5*Usp))
    diff = tr_chi_sp + 0.5 * Usp/U * n**2 - n

    return diff

def Uch_root_problem(Uch, chi0, n, U, docc):

    tr_chi_ch = chi_wk_trace(chi_wk_from_U_and_chi0_wk(chi0, U=-0.5*Uch))
    diff = tr_chi_ch - 2 * docc - n + n**2

    return diff

def solve_Usp_and_Uch(chi0, U, n, Usp0=0.1, Uch0=0.1):

    from scipy.optimize import fsolve
    
    Usp = fsolve(Usp_root_problem, Usp0, args=(chi0, n, U), xtol=1e-2)[0]
    docc = 0.25 * Usp / U * n**2        
    Uch = fsolve(Uch_root_problem, Uch0, args=(chi0, n, U, docc), xtol=1e-2)[0]

    return Usp, Uch, docc

In [ ]:
n = 1.0
Usp, Uch = 0.47, 0.37 # Initial guess

U_vec = np.concatenate((np.arange(0.3, 1., 0.2), np.arange(1., 6., 1.)))    
Usp_vec, Uch_vec, docc_vec = [np.zeros_like(U_vec) for x in range(3)]

print(''.join('| %-11s' % s for s in ['n', 'U', 'Usp', 'Uch', 'docc']), '|')
print('-'*67)

for idx, U in enumerate(U_vec):
    Usp, Uch, docc = solve_Usp_and_Uch(chi0_wk, U, n, Usp0=Usp, Uch0=Uch)
    Usp_vec[idx], Uch_vec[idx], docc_vec[idx] = Usp, Uch, docc
    print(''.join('| %4.4E ' % x for x in [n, U, Usp, Uch, docc]), '|')

In [ ]:
plt.figure(figsize=(3.25*2, 5))

plt.title(r'$\beta = %2.2f$' % beta)
plt.plot(U_vec, Usp_vec, 'o-', label=r'$U_{sp}$', alpha=0.5)
plt.plot(U_vec, Uch_vec, 'o-', label=r'$U_{ch}$', alpha=0.5)

plt.ylim([0, 20]); plt.xlim([0, 5])
plt.legend(loc='best'); plt.xlabel(r'$U$')
plt.tight_layout()
plt.savefig('figure_Usp_and_Uch_vs_U.pdf')

# TPSC and the Mermin-Wagner theorem

Temperature sweep for $U=4$

Spin structure factor 

$$S(k) \equiv \sum_n \chi_{sp}(i\omega_n, k)$$

In [ ]:
U = 4.

T_rpa_vec = np.concatenate((np.arange(10., 3., -1.), np.arange(3., 0.75, -0.2)))
S_rpa_vec = np.zeros_like(T_rpa_vec)

print(''.join('| %-11s' % s for s in ['T', 'beta', 'S_rpa']), '|')
print('-'*41)

for idx, T in enumerate(T_rpa_vec):

    beta = 1. / T
    g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
    chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)    

    chi_rpa_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, 0.5*U)
    
    S_rpa = chi_rpa_wk[:, k_pipi].data.sum().real # FIXME: Replace with .density() call
    S_rpa_vec[idx] = S_rpa
    
    print(''.join('| %4.4E ' % x for x in [T, beta, S_rpa]), '|')

In [ ]:
n = 1.
U = 4.

Usp, Uch = 1., 1. # initial guess

T_tpsc_vec = np.array([10., 9., 8., 7., 6., 5., 4., 3., 
                       2.5, 2.0, 1.5, 1.2, 1.0, 
                       0.8, 0.6, 0.4, 0.35, 0.3, 0.25])

S_tpsc_vec = np.zeros_like(T_tpsc_vec)
U_sp_vec = np.zeros_like(T_tpsc_vec)

print(''.join('| %-11s' % s for s in ['T', 'beta', 'Usp', 'Uch', 'docc', 'S_tpsc']), '|')
print('-'*80)

for idx, T in enumerate(T_tpsc_vec):

    beta = 1. / T    
    g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
    chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)    
    
    Usp, Uch, docc = solve_Usp_and_Uch(chi0_wk, U, n, Usp0=Usp, Uch0=Uch)
    
    chi_sp_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, 0.5*Usp)
    S_tpsc = chi_sp_wk[:, k_pipi].data.sum().real

    S_tpsc_vec[idx], U_sp_vec[idx] = S_tpsc, Usp

    print(''.join('| %4.4E ' % x for x in [T, beta, Usp, Uch, docc, S_tpsc]), '|')


In [ ]:
plt.figure(figsize=(3.25*2, 5*4))
subp = [4, 1, 1]

plt.subplot(*subp); subp[-1] += 1
plt.title(r'$U = %2.2f$' % U)
plt.plot(T_rpa_vec, S_rpa_vec, 'o-', label=r'$S_{RPA}$', alpha=0.5)
plt.plot(T_tpsc_vec, S_tpsc_vec, 'o-', label=r'$S_{TPSC}$', alpha=0.5)
plt.legend(loc='best')
plt.xlabel(r'$T$')

plt.subplot(*subp); subp[-1] += 1
plt.plot(T_rpa_vec, 1./S_rpa_vec, 'o-', alpha=0.5, label=r'$S_{RPA}^{-1}$')
plt.plot(T_tpsc_vec, 1./S_tpsc_vec, 'o-', alpha=0.5, label=r'$S_{TPSC}^{-1}$')
plt.legend(loc='best')
plt.xlabel(r'$T$'); plt.grid()

plt.subplot(*subp); subp[-1] += 1
plt.plot(T_rpa_vec, 1./S_rpa_vec, 'o-', alpha=0.5, label=r'$S_{RPA}^{-1}$')
plt.plot(T_tpsc_vec, 1./S_tpsc_vec, 'o-', alpha=0.5, label=r'$S_{TPSC}^{-1}$')
plt.legend(loc='best'); plt.xlim([0, 2]); plt.ylim([-0.1, 2.5])
plt.xlabel(r'$T$'); plt.grid()

plt.subplot(*subp); subp[-1] += 1
plt.plot(T_rpa_vec, U + 0*T_rpa_vec, 'o-', alpha=0.5, label=r'$U$')
plt.plot(T_tpsc_vec, U_sp_vec, 'o-', alpha=0.5, label=r'$U_{sp}$')
plt.legend(loc='best')
plt.xlabel(r'$T$')

plt.tight_layout()
plt.savefig('figure_S_TPSC_RPA_vs_T.pdf')

# RPA phase boundary

While the TPSC fulfills the Mermin-Wagner theorem RPA does not. Instead it predicts an anti-ferro magnetic transition at finite temperature for finite interaction $U$.


In [ ]:
Tc_rpa = np.arange(0.01, 1.0, 0.05)
Uc_rpa = np.zeros_like(Tc_rpa)

for idx, T in enumerate(Tc_rpa):

    beta = 1. / T    
    g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
    chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)    
        
    chi0_w0kpipi = chi0_wk[Idx(0), k_pipi][0, 0].real
    Uc = 2.0 / chi0_w0kpipi
    Uc_rpa[idx] = Uc
    
    print('T, beta, Uc =', T, beta, Uc)
        

In [ ]:
plt.plot(Uc_rpa, Tc_rpa, '.-')

# Extra excercise

Nearest neighbour $t$, next nearest neighbour $t'=0.175$, and next next nearest neighbour hopping $t''=0.05$, interaction $U = 5.75t$, and inverse temperature $\beta = 20$.

In [ ]:
beta = 20.
mu = 0.0

t = 1.0
tp = 0.175 * t
tpp = 0.05 * t

U = 5.75 * t

n_k = (8, 8, 1) 
n_w = 1024 * 4

In [ ]:
import numpy as np
from triqs.lattice.tight_binding import TBLattice

I = np.eye(2)

H_0 = TBLattice(
    units = [(1, 0, 0), (0, 1, 0)],
    hopping = {
        (+1, 0): -t * I,  
        (-1, 0): -t * I, 
        ( 0,+1): -t * I, 
        ( 0,-1): -t * I,
        # next nearest neighbour
        (+1, +1): -tp * I,  
        (+1, -1): -tp * I,  
        (-1, +1): -tp * I,  
        (-1, -1): -tp * I,  
        # next next nearest neighbour
        (+2, 0): -tpp * I,  
        (-2, 0): -tpp * I, 
        ( 0,+2): -tpp * I, 
        ( 0,-2): -tpp * I,
        },
    orbital_positions = [(0,0,0)]*2,
    orbital_names = ['up', 'do'],
    )

In [ ]:
kmesh = H_0.get_kmesh(n_k)
print('kmesh =\n', kmesh)
rmesh = H_0.get_rmesh(n_k)
print('rmesh =\n', rmesh)

e_k = H_0.on_gf_mesh_brillouin_zone(n_k)
print('e_k =\n', e_k)

In [ ]:
# Fixme move 2d plot to library

from k_space_viz import extend_data_on_boundary

data_k, k_vec, (kx, ky, kz) = extend_data_on_boundary(e_k[0, 0].data, n_k)
data_k = data_k[:, :, 0].real
extent_k = np.array([kx.min(), kx.max(), ky.min(), ky.max()])

from triqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.title(r'Dispersion $\epsilon_k$ and fermi surface contour $\epsilon_k = 0$')
plt.contour(kx, ky, data_k, levels=[0.])
plt.imshow(data_k, cmap=plt.get_cmap('RdBu_r'), origin='lower', 
           vmin=data_k.min(), vmax=data_k.max(),
           extent=extent_k)
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')
plt.colorbar(); plt.tight_layout()
plt.savefig('figure_e_k.pdf')
plt.show()

In [ ]:
from k_space_viz import k_space_path

G = np.array([0.0, 0.0, 0.0])
X = np.array([0.5, 0.5, 0.0])
M = np.array([0.5, 0.0, 0.0])

paths = [(G, X), (X, M), (M, G)]

k_path_vecs, k_plot, K_plot = k_space_path(paths, num=100)

from k_space_viz import get_rel_k_interpolator

interp = get_rel_k_interpolator(
    e_k[0, 0].data, kmesh, H_0.bz, n_k,
    extend_boundary=True, interpolator='linear2D')
e_plot = interp(k_path_vecs[:, :2])

from triqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.title(r'Dispersion $\epsilon_k$ along the high-symmetry path $\Gamma$-$X$-$M$-$\Gamma$')
plt.plot(k_plot, e_plot.real)

plt.grid(); plt.axes().set_xticks(K_plot)
plt.xlim([K_plot.min(), K_plot.max()])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$'])
plt.ylabel(r'$\epsilon_k$'); plt.tight_layout()
plt.savefig('figure_e_k_bandpath.pdf')
plt.show()

## TPSC calculation


In [ ]:
print('g0_wk')
g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
n = -g0_tr[Idx(0), Idx(0, 0, 0)].sum().real
print('n =', n)

In [ ]:
print('chi0_wk')
chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)

print('tpsc self cons')
Usp, Uch, docc = solve_Usp_and_Uch(chi0_wk, U, n, Usp0=Usp, Uch0=Uch)
print('Usp, Uch, docc =', Usp, Uch, docc)

In [ ]:
print('chi_sp_wk, chi_ch_wk')
chi_sp_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, +0.5*Usp)
chi_ch_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, -0.5*Uch)

In [ ]:
# todo: remove when we have the new tail

def fix_tr_tail(g_tr, r, gf_flag=True):
    g_t = g_tr[:, r]
    g_t.singularity = TailGf(g_t.target_shape)
    if gf_flag and np.linalg.norm(r.value) < 1e-9: g_t.singularity[1] = I
    return g_t

In [ ]:
target_shape = [1, 1]
wmesh = chi0_wk.mesh.components[0]
tmesh = chi0_tr.mesh.components[0]

chi_sp_wr = Gf(mesh=MeshProduct(wmesh, rmesh), target_shape=target_shape)
chi_sp_tr = Gf(mesh=MeshProduct(tmesh, rmesh), target_shape=target_shape)

chi_ch_wr = Gf(mesh=MeshProduct(wmesh, rmesh), target_shape=target_shape)
chi_ch_tr = Gf(mesh=MeshProduct(tmesh, rmesh), target_shape=target_shape)

print('k -> r')
for w in wmesh: 
    chi_sp_wr[w, :] = InverseFourier(chi_sp_wk[w, :])
    chi_ch_wr[w, :] = InverseFourier(chi_ch_wk[w, :])

print('w -> t')
for r in rmesh: 
    chi_sp_tr[:, r] = InverseFourier(fix_tr_tail(chi_sp_wr, r, gf_flag=False)) # Q: SHOULD WE FIX TAIL HERE??!!
    chi_ch_tr[:, r] = InverseFourier(fix_tr_tail(chi_ch_wr, r, gf_flag=False))

In [ ]:
plt.figure()
oplotr(chi_sp_tr[:, Idx(0, 0, 0)])
plt.figure()
oplotr(chi_sp_tr[:, Idx(1, 0, 0)])

In [ ]:
plt.figure()
oplotr(chi_ch_tr[:, Idx(0, 0, 0)])
plt.figure()
oplotr(chi_ch_tr[:, Idx(1, 0, 0)])

In [ ]:
# compute self energy

g_chi_sp_tr = bubble(g0_tr[0, 0], chi_sp_tr)
g_chi_ch_tr = bubble(g0_tr[0, 0], chi_ch_tr)

sigma_tr = U/8. * ( 3.*Usp*g_chi_sp_tr + Uch*g_chi_ch_tr )

print(sigma_tr)

In [ ]:
plt.figure()
oplotr(sigma_tr[:, Idx(0, 0, 0)])
plt.figure()
oplotr(sigma_tr[:, Idx(0, 0, 0)])
plt.xlim([19.5, 20.1])
plt.figure()
oplotr(sigma_tr[:, Idx(1, 0, 0)])

In [ ]:
target_shape = [1, 1]

sigma_wr = Gf(mesh=MeshProduct(wmesh, rmesh), target_shape=target_shape)
sigma_wk = Gf(mesh=MeshProduct(wmesh, kmesh), target_shape=target_shape)

for r in rmesh: sigma_wr[:, r] = Fourier(fix_tr_tail(sigma_tr, r, gf_flag=False))
for w in wmesh: sigma_wk[w, :] = Fourier(sigma_wr[w, :])

In [ ]:
# add hartree term
#for k in kmesh: sigma_wk[:, k] += U * 0.5 * n * I

In [ ]:
# Analytic continuation of Sigma_wk to w = 0 + i*epsilon

from triqs.gf import MeshReFreq

rwmesh = MeshReFreq(omega_min=-1., omega_max=1., n_max=10)
sigma_rwk = Gf(mesh=MeshProduct(rwmesh, kmesh), target_shape=[1, 1])

for k in kmesh:
    sigma_rw = sigma_rwk[:, k]
    sigma_rw.set_from_pade(sigma_wk[:, k], n_points=20)

In [ ]:
oplot(sigma_wk[:, Idx(0, 0, 0)])

In [ ]:
oplot(sigma_rwk[:, Idx(0, 0, 0)])

In [ ]:
# Plot G(w=0, k)
g0_rwk = Gf(mesh=MeshProduct(rwmesh, kmesh), target_shape=[1, 1])
g_rwk = Gf(mesh=MeshProduct(rwmesh, kmesh), target_shape=[1, 1])

for k in kmesh:
    eps = 5.e-1j
    g_rwk[:, k] << inverse(iOmega_n + eps - e_k[k][0, 0] - sigma_rwk[:, k])
    g0_rwk[:, k] << inverse(iOmega_n + eps - e_k[k][0, 0])    

In [ ]:
oplot(g0_rwk[:, Idx(0, 0, 0)])

In [ ]:
oplot(g_rwk[:, Idx(0, 0, 0)])

In [ ]:
g_k = g_rwk[Idx(5), :]
g0_k = g0_rwk[Idx(5), :]

In [ ]:
# Fixme move 2d plot to library

from k_space_viz import extend_data_on_boundary

data_k, k_vec, (kx, ky, kz) = extend_data_on_boundary(g_k.data, n_k)
data_k = - data_k[:, :, 0].imag / np.pi
extent_k = np.array([kx.min(), kx.max(), ky.min(), ky.max()])

from triqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.imshow(data_k, cmap=plt.get_cmap('RdBu_r'), origin='lower', 
           vmin=data_k.min(), vmax=data_k.max(),
           extent=extent_k)
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')
plt.colorbar(); plt.tight_layout()

In [ ]:
# Fixme move 2d plot to library

from k_space_viz import extend_data_on_boundary

data_k, k_vec, (kx, ky, kz) = extend_data_on_boundary(g0_k.data, n_k)
data_k = - data_k[:, :, 0].imag / np.pi
extent_k = np.array([kx.min(), kx.max(), ky.min(), ky.max()])

from triqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.imshow(data_k, cmap=plt.get_cmap('RdBu_r'), origin='lower', 
           vmin=data_k.min(), vmax=data_k.max(),
           extent=extent_k)
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')
plt.colorbar(); plt.tight_layout()